# Bront greedy Heuristic und MIP

Hier wollen wir die Korrektheit der Implementation der greedy heuristic (4.2.2) und der MIP formulation (4.2.1) aus Bront et al nachweisen.

In [3]:
import numpy as np
import pandas as pd

from gurobipy import *
import re

## Data

In [5]:
# example for 4.2.2 Greedy Heuristic
numProducts = n = 3
preference_weights = np.array([[1, 1, 1],
                               [0, 1, 0],
                               [0, 0, 1]])
preference_no_purchase = np.array([1, 1, 1])
w = revenues = np.array([100, 19, 19])
arrival_probability = np.array([0.2, 0.3, 0.5])

# unnecessary for this example
pi = 0

## Code

In [6]:
def column_MIP(pi, w = 0):  # pass w to test example for greedy heuristic
    L = len(preference_no_purchase)
    K = 1/min(preference_no_purchase.min(), np.min(preference_weights[np.nonzero(preference_weights)]))+1

    if isinstance(w, int) and w == 0:  # and is lazy version of &
        w = np.zeros_like(revenues, dtype=float)
        for j in np.arange(len(revenues)):
            w[j] = revenues[j] - sum(A[:, j]*pi)

    try:
        m = Model()

        mx = {}
        my = {}
        mz = {}

        # Variables
        for j in np.arange(numProducts):
            my[j] = m.addVar(0, 1, vtype=GRB.BINARY, name="y["+str(j)+"]")
        for l in np.arange(L):
            mx[l] = m.addVar(0.0, name="x["+str(l)+"]")
            temp = {}
            for j in np.arange(numProducts):
                temp[j] = m.addVar(0.0, name="z["+str(l)+","+str(j)+"]")
            mz[l] = temp

        # Objective TODO einfügen: arrival_probability[l]*
        m.setObjective(quicksum(w[j]*preference_weights[l, j]*mz[l][j]
                                for l in np.arange(L) for j in np.arange(numProducts)), GRB.MAXIMIZE)

        # Constraints
        mc1 = m.addConstrs((mx[l]*preference_no_purchase[l] + quicksum(preference_weights[l, i]*mz[l][i] for i in np.arange(numProducts)) == 1 for l in range(L)), name="mc1")
        mc2 = m.addConstrs((mx[l] - mz[l][i] <= K - K*my[i] for l in range(L) for i in range(numProducts)), name="mc2")
        mc3 = m.addConstrs((mz[l][i] <= mx[l] for l in range(L) for i in range(numProducts)), name="mc3")
        mc4 = m.addConstrs((mz[l][i] <= K*my[i] for l in range(L) for i in range(numProducts)), name="mc4")

        m.optimize()

        y = np.zeros_like(revenues)
        for j in np.arange(numProducts):
            y[j] = my[j].x

        return tuple(y), m.objVal

    except GurobiError:
        print('Error reported')

def column_greedy(pi, w = 0):  # pass w to test example for greedy heuristic
    L = len(preference_weights)

    # Step 1
    y = np.zeros_like(revenues)

    if isinstance(w, int) and w == 0:  # and is lazy version of &
        w = revenues - np.dot(A.transpose(),pi)  # calculate the opportunity costs

    # Step 2
    Sprime = set(np.where(w > 0)[0])

    # Step 3
    value_marginal = np.zeros_like(w, dtype=float)
    for j in Sprime:
        for l in np.arange(L):
            value_marginal[j] += preference_weights[l, j]/(preference_weights[l, j] + preference_no_purchase[l])
        value_marginal[j] *= w[j]
    jstar = np.argmax(value_marginal)
    v_new = value_marginal[jstar]

    S = {jstar}
    Sprime = Sprime-S

    # Step 4
    while True:
        v_akt = v_new
        v_temp = np.zeros_like(revenues, dtype=float)  # uses more space then necessary, but simplifies indices below
        for j in Sprime:
            for l in np.arange(L):
                z = 0
                n = 0
                for i in S.union({j}):
                    z += w[i]*preference_weights[l, i]
                    n += preference_weights[l, i] + preference_no_purchase[l]
                v_temp[j] += arrival_probability[l]*z/n
        jstar = np.argmax(value_marginal)
        v_new = value_marginal[jstar]
        if v_new > v_akt:
            S = S.union({jstar})
            Sprime = Sprime - {jstar}
        else:
            break

    # Step 5
    y[list(S)] = 1
    return tuple(y), v_new

In [14]:
print("--------------------")
print("Greedy Optimization")
print("--------------------\n")
print(column_greedy(pi, w))

print("\n\n--------------------")
print("MIP Optimization")
print("--------------------\n")
print(column_MIP(pi, w))

--------------------
Greedy Optimization
--------------------

((1, 0, 0), 50.0)


--------------------
MIP Optimization
--------------------

Optimize a model with 30 rows, 15 columns and 71 nonzeros
Variable types: 12 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [2e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Presolve removed 8 rows and 4 columns
Presolve time: 0.00s
Presolved: 22 rows, 11 columns, 57 nonzeros
Variable types: 4 continuous, 7 integer (3 binary)

Root relaxation: objective 5.950000e+01, 8 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   59.50000    0    7          -   59.50000      -     -    0s
H    0     0                      53.5000000   59.50000  11.2%     -    0s
     0     0     cutoff    0        53.50000   53.50000  0.00%